In [1]:
import os
import time

import torch

from juju_tools.utils import *

from juju_tools.models.loading_utils import ModelLoader
from juju_tools.models.llama3 import Tokenizer, ChatFormat

from transformers import (
    AutoTokenizer,
)

2024-06-28 17:34:03,779 - bitsandbytes.cextension - DEBUG -> Loading bitsandbytes native library from: c:\Users\volta\AppData\Local\Programs\Python\Python310\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


c:\Users\volta\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llama_loader = ModelLoader(name="llama3-8b", weight_path="llama3_8b_weights", quant_type="nf4")
config = llama_loader.config

2024-06-28 17:34:05,879 - root - INFO ->  Model has been loaded... 
2024-06-28 17:34:05,880 - root - INFO -> Quantizing Model Layers... 
2024-06-28 17:34:05,881 - root - INFO -> ('layers', 'embed', 'cls_head', 'pre_norm')
2024-06-28 17:34:05,881 - root - INFO -> ('gq_attn', 'ffn', 'attn_prenorm', 'ffn_prenorm')
2024-06-28 17:34:05,881 - root - INFO -> Quantizing block: 0 
2024-06-28 17:34:07,211 - root - INFO -> Quantizing block: 1 
2024-06-28 17:34:08,582 - root - INFO -> Quantizing block: 2 
2024-06-28 17:34:09,946 - root - INFO -> Quantizing block: 3 
2024-06-28 17:34:11,218 - root - INFO -> Quantizing block: 4 
2024-06-28 17:34:12,516 - root - INFO -> Quantizing block: 5 
2024-06-28 17:34:13,750 - root - INFO -> Quantizing block: 6 
2024-06-28 17:34:15,059 - root - INFO -> Quantizing block: 7 
2024-06-28 17:34:16,375 - root - INFO -> Quantizing block: 8 
2024-06-28 17:34:17,705 - root - INFO -> Quantizing block: 9 
2024-06-28 17:34:18,980 - root - INFO -> Quantizing block: 10 
2024

In [3]:
def sample_top_p(probs, p):
    probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    mask = probs_sum - probs_sort > p
    probs_sort[mask] = 0.0
    probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
    next_token = torch.multinomial(probs_sort, num_samples=1)
    next_token = torch.gather(probs_idx, -1, next_token)
    return next_token



In [4]:
# @torch.inference_mode()
# def generate(
#     prompt_tokens: List[List[int]],
#     max_gen_len: int,
#     temperature: float = 0.6,
#     top_p: float = 0.9,
#     logprobs: bool = False,
#     echo: bool = False,
# ) -> Tuple[List[List[int]], Optional[List[List[float]]]]:
#     """
#     Generate text sequences based on provided prompts using the language generation model.

#     Args:
#         prompt_tokens (List[List[int]]): List of tokenized prompts, where each prompt is represented as a list of integers.
#         max_gen_len (int): Maximum length of the generated text sequence.
#         temperature (float, optional): Temperature value for controlling randomness in sampling. Defaults to 0.6.
#         top_p (float, optional): Top-p probability threshold for nucleus sampling. Defaults to 0.9.
#         logprobs (bool, optional): Flag indicating whether to compute token log probabilities. Defaults to False.
#         echo (bool, optional): Flag indicating whether to include prompt tokens in the generated output. Defaults to False.

#     Returns:
#         Tuple[List[List[int]], Optional[List[List[float]]]]: A tuple containing generated token sequences and, if logprobs is True, corresponding token log probabilities.

#     Note:
#         This method uses the provided prompts as a basis for generating text. It employs nucleus sampling to produce text with controlled randomness.
#         If logprobs is True, token log probabilities are computed for each generated token.
    
#     """

#     tokenizer = Tokenizer("C:\\Users\\volta\Documents\\GitHub\\z-git\\j-tools\\juju_tools\\models\\llama3\\tokenizer.model")


#     config = llama_loader.get_config()
#     bsz = len(prompt_tokens)
#     assert bsz <= config.bsz, (bsz, config.bsz)

#     min_prompt_len = min(len(t) for t in prompt_tokens)
#     max_prompt_len = max(len(t) for t in prompt_tokens)
#     assert max_prompt_len <= config.max_tokens

#     total_len = min(config.max_tokens, max_gen_len + max_prompt_len)

#     pad_id = tokenizer.pad_id
#     tokens = torch.full((bsz, total_len), pad_id, dtype=torch.long, device="cuda")
#     for k, t in enumerate(prompt_tokens):
#         tokens[k, : len(t)] = torch.tensor(t, dtype=torch.long, device="cuda")
#     if logprobs:
#         token_logprobs = torch.zeros_like(tokens, dtype=torch.float)

#     prev_pos = 0
#     eos_reached = torch.tensor([False] * bsz, device="cuda")
#     input_text_mask = tokens != pad_id

#     stop_tokens = torch.tensor(list(tokenizer.stop_tokens))

#     for cur_pos in range(min_prompt_len, total_len):
#         logits = llama_loader.get_model()(tokens[:, prev_pos:cur_pos], shift=prev_pos)
#         if temperature > 0:
#             probs = torch.softmax(logits[:, -1] / temperature, dim=-1)
#             next_token = sample_top_p(probs, top_p)
#         else:
#             next_token = torch.argmax(logits[:, -1], dim=-1)

#         next_token = next_token.reshape(-1)
#         # only replace token if prompt has already been generated
#         next_token = torch.where(
#             input_text_mask[:, cur_pos], tokens[:, cur_pos], next_token
#         )
#         tokens[:, cur_pos] = next_token
#         if logprobs:
#             token_logprobs[:, prev_pos + 1 : cur_pos + 1] = -F.cross_entropy(
#                 input=logits.transpose(1, 2),
#                 target=tokens[:, prev_pos + 1 : cur_pos + 1],
#                 reduction="none",
#                 ignore_index=pad_id,
#             )
#         eos_reached |= (~input_text_mask[:, cur_pos]) & (
#             torch.isin(next_token, stop_tokens.cuda())
#         )
        
#         for lool in range(tokens.shape[0]):
#             gen_toks = tokens[lool, : cur_pos + 1].detach().cpu().numpy().tolist()
#             logger.info(f"{tokenizer.decode(gen_toks)}")

#         prev_pos = cur_pos
#         if all(eos_reached):
#             break

#     if logprobs:
#         token_logprobs = token_logprobs.tolist()
#     out_tokens, out_logprobs = [], []
#     for i, toks in enumerate(tokens.tolist()):
#         # cut to max gen len
#         start = 0 if echo else len(prompt_tokens[i])
#         toks = toks[start : len(prompt_tokens[i]) + max_gen_len]
#         probs = None
#         if logprobs:
#             probs = token_logprobs[i][start : len(prompt_tokens[i]) + max_gen_len]
#         # cut to after eos tok if any
#         for stop_token in tokenizer.stop_tokens:
#             try:
#                 eos_idx = toks.index(stop_token)
#                 toks = toks[:eos_idx]
#                 probs = probs[:eos_idx] if logprobs else None
#             except ValueError:
#                 pass
#         out_tokens.append(toks)
#         out_logprobs.append(probs)
#     return (out_tokens, out_logprobs if logprobs else None)


In [5]:
# llama_loader.get_model().clear_cache()

# tokenizer = Tokenizer("C:\\Users\\volta\Documents\\GitHub\\z-git\\j-tools\\juju_tools\\models\\llama3\\tokenizer.model")

# cf = ChatFormat(tokenizer)
# dialogs = [
#         # [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
#         [
#             {"role": "user", "content": "I am going to Paris, what should I see?"},
#             {
#                 "role": "assistant",
#                 "content": """\
# Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

# 1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
# 2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
# 3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

# These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
#             },
#             {"role": "user", "content": "What is so great about #1?"},
#         # ],
#         # [
#         #     {"role": "system", "content": "Always answer with Haiku"},
#         #     {"role": "user", "content": "I am going to Paris, what should I see?"},
#         # ],
#         # [
#         #     {
#         #         "role": "system",
#         #         "content": "Always answer with emojis",
#         #     },
#         #     {"role": "user", "content": "How to go from Beijing to NY?"},
#         ],
#     ]

# prompt_tokens = [cf.encode_dialog_prompt(dialog) for dialog in dialogs]

# generate(prompt_tokens=prompt_tokens, temperature=0, max_gen_len=120)


In [8]:
prev_tok = 0

@torch.inference_mode()
def run_local_model(message: list[dict]) -> None:
    global prev_tok
    with torch.no_grad():

        gen_len = 120

        tok = Tokenizer("C:\\Users\\volta\Documents\\GitHub\\z-git\\j-tools\\juju_tools\\models\\llama3\\tokenizer.model")

        cf = ChatFormat(tok)

        # y = "The meaning of life is to"  # torch.randint(10, (2, 10)).cuda()
        # y = [tok.encode(s=y, bos=True, eos=False)]

        logger.info(f"{message}")
        
        y = [cf.encode_dialog_prompt(message)]

        tokens = torch.full((1, len(y[0]) + gen_len), fill_value=config.pad_tok_idx,
                            dtype=torch.long, device=config.device)

        for idx, t in enumerate(y):
            tokens[idx, :len(t)] = torch.tensor(t, dtype=torch.long, device=config.device)

        text_mask = tokens != config.pad_tok_idx

        stops = torch.tensor(list(tok.stop_tokens))

        

        start = time.time()

        for curr_tok in range(len(y[0]) + prev_tok, gen_len + len(y[0]) + prev_tok):

            logger.info(f"Current Token Being Generated {curr_tok} | {tokens[..., prev_tok:curr_tok]}")

            logits = llama_loader.get_model()(input=tokens[:, prev_tok:curr_tok], shift=prev_tok)

            probs = torch.softmax(logits[:, -1] / .6, dim=-1)
            next_tok = sample_top_p(probs, .9)  # torch.argmax(logits[:, -1], dim=-1).reshape(-1) #

            next_tok = torch.where(text_mask[:, curr_tok], tokens[:, curr_tok], next_tok)

            tokens[:, curr_tok] = next_tok

            logger.info(f"TOKEN: {next_tok}")

            gen_toks = tokens[0, : curr_tok + 1].detach().cpu().numpy().tolist()

            logger.info(f"{tok.decode(gen_toks)}")
            prev_tok = curr_tok

            if torch.isin(next_tok.cpu(), stops):
                break

        logger.info(f"Computation time: {time.time() - start} seconds")





def main(**kwargs) -> Optional[Tensor]:
    return run_local_model(**kwargs)

In [9]:
if __name__ == "__main__":
    logger.info("Starting task... ")
    
    # llama_loader.get_model().clear_cache()

    message = [
            {"role": "system", "content": "You are a smart chatbot."},
            {"role": "user", "content": "Write me a World War II essay, please, thank you."},
        ]

    main(message=message)
    # get_hf_weights()
    logger.info("...Finished")

2024-06-28 17:35:18,726 - root - INFO -> Starting task... 
2024-06-28 17:35:18,983 - juju_tools.models.llama3.llama_tokenizer - INFO -> Reloaded tiktoken model from C:\Users\volta\Documents\GitHub\z-git\j-tools\juju_tools\models\llama3\tokenizer.model
2024-06-28 17:35:18,984 - juju_tools.models.llama3.llama_tokenizer - INFO -> #words: 128256 - BOS ID: 128000 - EOS ID: 128001
2024-06-28 17:35:18,985 - root - INFO -> [{'role': 'user', 'content': "It's on World War II mate."}]
2024-06-28 17:35:19,008 - root - INFO -> Current Token Being Generated 18 | tensor([[128000, 128006,    882, 128007,    271,   2181,    596,    389,   4435,
           5111,   8105,  30276,     13, 128009, 128006,  78191, 128007,    271]],
       device='cuda:0')
2024-06-28 17:35:23,140 - root - INFO -> TOKEN: tensor([[5028]], device='cuda:0')
2024-06-28 17:35:23,142 - root - INFO -> <|begin_of_text|><|start_header_id|>user<|end_header_id|>

It's on World War II mate.<|eot_id|><|start_header_id|>assistant<|end_heade

In [ ]:
# from juju_tools.models.llama3.get_llama3_hf import save_model_weights
# import os
# import time

# import torch

# from juju_tools.utils import *

# from juju_tools.models.loading_utils import ModelLoader
# from juju_tools.models.llama3 import Tokenizer, ChatFormat

# def get_hf_weights() -> None:
#     logger.info(f"{os.getcwd()}")
#     hf_path = input("Enter the local hugging face path: ")
#     save_model_weights(hf_path, str(os.getcwd()) + "\\")
    
# get_hf_weights()